Install all needed libraries

In [101]:
import csv
import numpy as np
import scipy as sci
from math import *
import pandas as pd
from random import shuffle
import scipy.spatial
import xml

I have shared the datafile with this python notebook. You can upload the data file using the Files features in the left options bar.

In [102]:
PBDB_data= "beta Cambrian Bivalvia_data.csv" #uploading data

In [103]:
Fields=[]
Observations=[]

In [104]:
# reading csv file
with open(PBDB_data, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
    
      
    # extracting field names through first row
    Fields = next(csvreader)
  
    parsed_csv = list(csvreader)

    # extracting each data row one by one
    for Observation in parsed_csv:
        Observations.append(Observation)
  
    # get total number of rows
    print("Total no. of rows (Including the title row): %d"%(csvreader.line_num))
    # get Titles from the columns
    print('Field names are:' + ', '.join(Field for Field in Fields))

Total no. of rows (Including the title row): 83
Field names are:accepted_name, lithdescript, lithology1, environment, geology_comments


In [105]:
#seperating all the lists into indiviual list
data = pd.read_csv("beta Cambrian Bivalvia_data.csv")

name = data['accepted_name'].tolist()
LithDescription = data['lithdescript'].tolist()
lithology = data['lithology1'].tolist()
Enivornment = data['environment'].tolist()
more_comments = data['geology_comments'].tolist()

#print('Species Names:', name)
#print('Lithology:',lithology)
#print('Enivornment:', Enivornment)
#print('Comments about lithology', more_comments)
#print('Decription of lithology:', LithDescription)

In [106]:
#get unique values from each list

def unique_values(counts):
    unique = []
    for count in counts:
        if count not in unique:
            unique.append(count)
    return unique


Uni_litho=unique_values(lithology) #get uniques values from lithology list
Uni_Species=unique_values(name)
Uni_Enivornment=unique_values(Enivornment)
print(Uni_litho)
print(Uni_Species)
print(Uni_Enivornment)

['wackestone', '"limestone"', '"shale"', 'siltstone', 'dolomite', 'packstone', 'framestone', 'lime mudstone']
['Fordilla troyensis', 'Fordilla germanica', 'Pojetaia runnegari', 'Pojetaia dentifera']
['peritidal', 'carbonate indet.', 'marine indet.', 'offshore', 'slope', 'deep subtidal indet.', 'shallow subtidal indet.', 'reef, buildup or bioherm']


In [107]:
#get Preence-Abscence Matrix for the data with actual no. of occurences

Samples_L= np.arange(0,len(lithology))
dict = {"Indiviual": Samples_L,'Species': name, 'Lithology': lithology, 'Enivornment': Enivornment}

df1 = pd.DataFrame(dict) #creating panda dataframe of the data
    
df1
#print(df1)
df1['Species'] = df1['Indiviual'].map(df1.set_index('Indiviual')['Species'])
g = df1.groupby('Species')['Lithology'].apply(list).reset_index()

g_sample= g.join(pd.get_dummies(g['Lithology']\
 .apply(pd.Series).stack()).sum(level=0)).drop('Lithology', 1)

g_sample
#print(dict)

,Species,"""limestone""","""shale""",dolomite,framestone,lime mudstone,packstone,siltstone,wackestone
0,Fordilla germanica,1,0,0,0,0,0,0,0
1,Fordilla troyensis,3,1,0,0,0,0,0,1
2,Pojetaia dentifera,0,0,0,0,1,0,0,0
3,Pojetaia runnegari,43,4,2,2,0,15,1,8


In [108]:
ab2 = g_sample.iloc[:,1:]
ab1 = ab2.to_numpy()
print(ab1)
#print(ab2)

[[ 1  0  0  0  0  0  0  0]
 [ 3  1  0  0  0  0  0  1]
 [ 0  0  0  0  1  0  0  0]
 [43  4  2  2  0 15  1  8]]


In [109]:
#get Presence-Abscence matrix with Binary values (Boolean values)
#IF the species is present then the value is 1 and if absent , then 0

ab = g_sample
ab.loc[ab['"shale"']>1,'"shale"']=1
ab.loc[ab['"limestone"']>1,'"limestone"']=1
ab.loc[ab['dolomite']>1,'dolomite']=1
ab.loc[ab['framestone']>1,'framestone']=1
ab.loc[ab['lime mudstone']>1,'lime mudstone']=1
ab.loc[ab['packstone']>1,'packstone']=1
ab.loc[ab['siltstone']>1,'siltstone']=1
ab.loc[ab['wackestone']>1,'wackestone']=1

ab
ab2_pa = ab.iloc[:,1:]
ab1_pa = ab2_pa.to_numpy()
print(ab1_pa)
#print(ab2)

[[1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]
 [1 1 1 1 0 1 1 1]]


In [110]:
#get Bray-Curtis Disimilarity number for the data


Pre_abs_matrix_bray_curtis = np.ones([8,8])

q=0
s=0
for q in range(len(ab1[0])):
  for s in range(len(ab1[0])):
    #print([row[q] for row in ab1],[row[s] for row in ab1])
    Bray_Curtis_number = scipy.spatial.distance.braycurtis([row[q] for row in ab1],[row[s] for row in ab1])
    Pre_abs_matrix_bray_curtis[s,q] = Bray_Curtis_number

print(Pre_abs_matrix_bray_curtis)


[[0.         0.80769231 0.91836735 0.91836735 1.         0.51612903
  0.95833333 0.67857143]
 [0.80769231 0.         0.42857143 0.42857143 1.         0.6
  0.66666667 0.28571429]
 [0.91836735 0.42857143 0.         0.         1.         0.76470588
  0.33333333 0.63636364]
 [0.91836735 0.42857143 0.         0.         1.         0.76470588
  0.33333333 0.63636364]
 [1.         1.         1.         1.         0.         1.
  1.         1.        ]
 [0.51612903 0.6        0.76470588 0.76470588 1.         0.
  0.875      0.33333333]
 [0.95833333 0.66666667 0.33333333 0.33333333 1.         0.875
  0.         0.8       ]
 [0.67857143 0.28571429 0.63636364 0.63636364 1.         0.33333333
  0.8        0.        ]]


The Bray-Curtis dissimilarity is always a number between 0 and 1. 
If 0, the two sites share all the same species; if 1, they don’t share any species.

In [111]:
Pre_abs_matrix_jaccard = np.ones([8,8])

q=0
s=0
for q in range(len(ab1_pa[0])):
  for s in range(len(ab1_pa[0])):
    jaccard_number = scipy.spatial.distance.jaccard([row[q] for row in ab1_pa],[row[s] for row in ab1_pa]) # 
    Pre_abs_matrix_jaccard[s,q] = (1-jaccard_number)*100

print(Pre_abs_matrix_jaccard)

[[100.          66.66666667  33.33333333  33.33333333   0.
   33.33333333  33.33333333  66.66666667]
 [ 66.66666667 100.          50.          50.           0.
   50.          50.         100.        ]
 [ 33.33333333  50.         100.         100.           0.
  100.         100.          50.        ]
 [ 33.33333333  50.         100.         100.           0.
  100.         100.          50.        ]
 [  0.           0.           0.           0.         100.
    0.           0.           0.        ]
 [ 33.33333333  50.         100.         100.           0.
  100.         100.          50.        ]
 [ 33.33333333  50.         100.         100.           0.
  100.         100.          50.        ]
 [ 66.66666667 100.          50.          50.           0.
   50.          50.         100.        ]]


The output values are in percentage

100 % - they share same species
50 % - two enivornment share half species
0 % - Dont sare any kind of species

In [112]:
#DIFFERENT ENIVORNMENTS

#get Preence-Abscence Matrix for the data with actual no. of occurences

Samples_L= np.arange(0,len(lithology))
dict = {"Indiviual": Samples_L,'Species': name, 'Lithology': lithology, 'Enivornment': Enivornment}

df1 = pd.DataFrame(dict) #creating panda dataframe of the data
    
df1
#print(df1)
df1['Species'] = df1['Indiviual'].map(df1.set_index('Indiviual')['Species'])
g = df1.groupby('Species')['Enivornment'].apply(list).reset_index()

g_sample_2= g.join(pd.get_dummies(g['Enivornment']\
 .apply(pd.Series).stack()).sum(level=0)).drop('Enivornment', 1)

g_sample_2
#print(dict)

,Species,carbonate indet.,deep subtidal indet.,marine indet.,offshore,peritidal,"reef, buildup or bioherm",shallow subtidal indet.,slope
0,Fordilla germanica,0,0,0,0,1,0,0,0
1,Fordilla troyensis,3,0,1,0,1,0,0,0
2,Pojetaia dentifera,1,0,0,0,0,0,0,0
3,Pojetaia runnegari,59,2,3,1,0,8,1,1


In [113]:
ab2 = g_sample_2.iloc[:,1:]
ab1 = ab2.to_numpy()
#print(ab1)
#print(ab2)


In [114]:
#get Presence-Abscence matrix with Binary values (Boolean values)
#IF the species is present then the value is 1 and if absent , then 0

ab = g_sample_2
ab.loc[ab['peritidal']>1,'peritidal']=1
ab.loc[ab['carbonate indet.']>1,'carbonate indet.']=1
ab.loc[ab['marine indet.']>1,'marine indet.']=1
ab.loc[ab['offshore']>1,'offshore']=1
ab.loc[ab['slope']>1,'slope']=1
ab.loc[ab['deep subtidal indet.']>1,'deep subtidal indet.']=1
ab.loc[ab['shallow subtidal indet.']>1,'shallow subtidal indet.']=1
ab.loc[ab['reef, buildup or bioherm']>1,'reef, buildup or bioherm']=1

ab
ab2_pa_2 = ab.iloc[:,1:]
ab1_pa_2 = ab2_pa_2.to_numpy()
print(ab1_pa)
#print(ab2)


[[1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]
 [1 1 1 1 0 1 1 1]]


In [115]:
#get Bray-Curtis Disimilarity number for the data


Pre_abs_matrix_bray_curtis_2 = np.ones([8,8])

q=0
s=0
for q in range(len(ab1[0])):
  for s in range(len(ab1[0])):
    #print(q)
    Bray_Curtis_number = scipy.spatial.distance.braycurtis([row[q] for row in ab1],[row[s] for row in ab1])
    Pre_abs_matrix_bray_curtis_2[s,q] = Bray_Curtis_number

print(Pre_abs_matrix_bray_curtis_2)

[[0.         0.93846154 0.88059701 0.96875    0.96923077 0.77464789
  0.96875    0.96875   ]
 [0.93846154 0.         0.33333333 0.33333333 1.         0.6
  0.33333333 0.33333333]
 [0.88059701 0.33333333 0.         0.6        0.66666667 0.5
  0.6        0.6       ]
 [0.96875    0.33333333 0.6        0.         1.         0.77777778
  0.         0.        ]
 [0.96923077 1.         0.66666667 1.         0.         1.
  1.         1.        ]
 [0.77464789 0.6        0.5        0.77777778 1.         0.
  0.77777778 0.77777778]
 [0.96875    0.33333333 0.6        0.         1.         0.77777778
  0.         0.        ]
 [0.96875    0.33333333 0.6        0.         1.         0.77777778
  0.         0.        ]]


In [116]:
Pre_abs_matrix_jaccard_2 = np.ones([8,8])

q=0
s=0
for q in range(len(ab1_pa_2[0])):
  for s in range(len(ab1_pa_2[0])):
    jaccard_number_2 = scipy.spatial.distance.jaccard([row[q] for row in ab1_pa_2],[row[s] for row in ab1_pa_2]) 
    Pre_abs_matrix_jaccard_2[s,q] = (1-jaccard_number_2)*100

print(Pre_abs_matrix_jaccard_2)

[[100.          33.33333333  66.66666667  33.33333333  25.
   33.33333333  33.33333333  33.33333333]
 [ 33.33333333 100.          50.         100.           0.
  100.         100.         100.        ]
 [ 66.66666667  50.         100.          50.          33.33333333
   50.          50.          50.        ]
 [ 33.33333333 100.          50.         100.           0.
  100.         100.         100.        ]
 [ 25.           0.          33.33333333   0.         100.
    0.           0.           0.        ]
 [ 33.33333333 100.          50.         100.           0.
  100.         100.         100.        ]
 [ 33.33333333 100.          50.         100.           0.
  100.         100.         100.        ]
 [ 33.33333333 100.          50.         100.           0.
  100.         100.         100.        ]]
